Goal of analysis script is to pull in Med-PC files from Pavlovian conditioning experiment and pull out event times and stamps into a df

Import numpy

In [1]:
import numpy as np
import pandas as pd
import re

Open dialog box and prompt user to select directory containing the data files

In [2]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav7


Use os module to create list of data folders

In [3]:
import os
session_list = os.listdir(root.directory)
session_list

['180628_266_Pav7',
 '180628_268_Pav7',
 '180628_269_Pav7',
 '180628_270_Pav7',
 '180628_271_Pav7',
 '180628_273_Pav7',
 '180628_274_Pav7',
 '180628_275_Pav7',
 '180628_277_Pav7',
 '180628_278_Pav7',
 '180628_280_Pav7',
 '180628_283_Pav7',
 '180628_287_Pav7',
 '180628_288_Pav7',
 '180628_289_Pav7',
 '180628_290_Pav7',
 '180628_292_Pav7',
 '180629_Pav7_267',
 '180629_Pav7_282',
 '180629_Pav7_291']

In [4]:
file_paths = []
for file in session_list:
    file_path = root.directory + '/' + file
    file_paths.append(file_path)

file_paths[0]

'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav7/180628_266_Pav7'

In [51]:
def get_file_info(file_path_name):
    split = file_path_name.split('/')
    date, animal_number, session = split[12].split('_')
    
    return date, animal_number, session

In [6]:
def make_array_from_path (file_path_name):
    data = pd.read_table(file_path_name)
    data_table = pd.DataFrame(data = data)
    data_array = data_table.values
    
    return data_array

In [7]:
def get_eandt(array):
    event_letter = "E"
    time_letter = "T"
    re_l = r"..[A-Z]...$"
    re_n = r'.. '

    i = 0
    j = 0
    k = 0
    event_data = []
    time_data = []
    for row in array:
        if re.search(re_l, str(row)):
            marker = str(row) 
            j = i
            k = i
            if re.match(marker, event_letter):
                j = j + 1
                while re.search(re_n, str(array[j])):
                    event_data.append((str(array[j])))
                    j = j + 1
            if re.match(marker, time_letter):
                k = k + 1
                while re.search(re_n, str(array[k])):
                    time_data.append((str(array[k])))
                    k = k + 1
            else:
                i = i + 1
        else:
            i = i + 1
            
    re_n = r'^\d(\d|\.)(\d|\.)(\d|\.)'
    event_data_list = []
    for row in event_data:
        data_set_split = row.split(" ")
        for line in data_set_split:
            if re.search(re_n, line):
                event_data_list.append(float(line[0:(len(line)-2)]))
                
    re_n = r'^\d(\d|\.)(\d|\.)(\d|\.)'
    time_data_list = []
    for row in time_data:
        data_set_split = re.split(" |'", row)
        for line in data_set_split:
            if re.search(re_n, line):
                time_data_list.append(float(line[0:(len(line)-1)]))

    eandt = np.column_stack((event_data_list, time_data_list))
    
    return eandt

Make list of CS and ITI start and end times

\  Event stamps ( array E )
\   1   = Rt lever press/NP
\   2   = Lt lever press/NP
\   5   = Pellet reinforcement delivery
\   6   = Food tray head entry
\   7   = Right cue light on
\   8   = Left cue light on
\   9   = Right cue light off
\   10  = Left cue ight off
\   11  = Magazine tray light on
\   12  = Magazine tray light off
\   13  = Liquid delivery turns on
\   14  = Liquid delivery turns off
\   15  = ITI starts
\   16  = ITI ends
\   18  = Right lever extends
\   19  = Left lever extends
\   20  = Right lever retracts
\   21  = Left lever retracts
\   22  = CS on
\   23  = CS off
\   100 = Session termination
\   999 = Place holder - no event

In [8]:
def get_cs_and_iti(es_et):
    cs_start = es_et[es_et.event_stamp == 22].reset_index()
    cs_end = es_et[es_et.event_stamp == 23].reset_index()
    ITI_start = es_et[es_et.event_stamp == 15].reset_index()
    ITI_end = es_et[es_et.event_stamp == 16].reset_index()
    return cs_start, cs_end, ITI_start, ITI_end

we routinely measure 
(a) the number of lever deflections and food cup head entries during the CS period, 
count number of lever pressess, count number of food cup head entries

(b) the probability of contacting the lever or
entering the food cup during the CS period on each trial (defined
as the number of trials with a lever press or food cup entry, divided
by the total number of trials). count numnber of trials a lp or he occured then divide by number of trials

(c) the ave latency to the first lever
deflection or food cup entry. find latency for first lp or he for each trial then average over number of trials with lp of he

Using these measures we devised a
‘‘PCA Score’’, as follows. From the CS lever deflections and CS
food cup entries, we calculated three measures of approach: 

1) Response Bias (ratio of lever presses and food cup entries in relation
to total number of responses), 

2) Probability Difference (the difference
between the probability of pressing the lever and the probability of
entering the food cup), and 

3) Latency Score (difference between the
latencies to approach the lever and the food cup). 

Averaging these
three measures of approach produces a PCA Score for each
individual, on each day of training. 

Scores derived this way range
from 21 to +1, whereby scores of 21 and +1 indicate strong
biases toward goal-tracking and sign-tracking, respectively, and a
score of zero indicates that the two responses are equally
distributed. Table 1 describes these calculations in detail. 

We also
measured the number of food cup head entries during the intertrial
interval (ITI), but these are not included as part of the PCA
Score.

In [9]:
def PCA_analysis(cs_iti, es_et):
    PCA_values = pd.DataFrame()

    lever = 1
    lever_press = 0
    head_entries = 0
    lever_prob = 0
    head_prob = 0
    ave_lat_lever = 0
    ave_lat_head = 0

    count_lever = 0
    count_head = 0
    lat_lever_sum = 0
    lat_head_sum = 0

    response_bias = 0
    prob_diff = 0
    lat_score = 0 
    PCA_score = 0

    i = 0

    while i < 25:
        data = es_et[(es_et.time_stamp >= cs_iti.CS_start[i]) & (es_et.time_stamp <= cs_iti.CS_end[i])]
    
        if len(data[data.event_stamp == 1]) > 0:
            lever == 1
        if len(data[data.event_stamp == 2]) > 0:
            lever == 2
        
        lever_press = lever_press + len(data[data.event_stamp == lever])
    
        head_entries = head_entries + len(data[data.event_stamp == 6])

        if len(data[data.event_stamp == lever]) > 0:
            count_lever = count_lever + 1
            lat_lever_sum = lat_lever_sum + data[data.event_stamp == lever].time_stamp.iloc[0]
        
        if len(data[data.event_stamp == 6]) > 0:
            count_head = count_head + 1
            lat_head_sum = lat_head_sum + data[data.event_stamp == 6].time_stamp.iloc[0]
        
        i = i + 1
    
    lever_prob = count_lever / 25
    head_prob = count_head / 25

    if lat_lever_sum > 0:
        ave_lat_lever = lat_lever_sum / count_lever * .00225
    if lat_head_sum > 0:
        ave_lat_head = lat_head_sum / count_head * .00225

    response_bias = (lever_press - head_entries) / (lever_press + head_entries)
    prob_diff = lever_prob - head_prob
    lat_score = (ave_lat_lever - ave_lat_head) / 10

    PCA_score = (response_bias + prob_diff + lat_score) / 3

    PCA_values['lever_press'] = [lever_press]
    PCA_values['head_entries'] = head_entries
    PCA_values['lever_prob'] = lever_prob
    PCA_values['head_prob'] = head_prob
    PCA_values['ave_lat_lever'] = ave_lat_lever
    PCA_values['ave_lat_head'] = ave_lat_head
    PCA_values['response_bias'] = response_bias
    PCA_values['prob_diff'] = prob_diff
    PCA_values['PCA_score'] = PCA_score

    return PCA_values

In [53]:
def analyze_PCA_from_med(file_path):
    date = 0
    animal_number = 0
    session = 0
    #get file info
    date, animal_number, session = get_file_info(file_path)
    #make array from file path
    array = make_array_from_path(file_path)
    #get event times and event stamps
    es_et = pd.DataFrame(data = get_eandt(array))
    es_et.columns = ['event_stamp', 'time_stamp']
    #get cs and iti start and end times
    cs_start, cs_end, ITI_start, ITI_end = get_cs_and_iti(es_et)
    #Make list into dataframe CS and ITI start and end times
    cs_iti = pd.DataFrame(data = [cs_start.time_stamp, cs_end.time_stamp, ITI_start.time_stamp[:-1], ITI_end.time_stamp])
    cs_iti = cs_iti.T
    cs_iti.columns = ['CS_start', 'CS_end', 'ITI_start', 'ITI_end']
    #Use dataframe of start and end times to select events during each CS and compute PCA parameters
    PCA_values = PCA_analysis(cs_iti, es_et)
    PCA_values['Date'] = date
    PCA_values['Animal'] = animal_number
    PCA_values['Session'] = session
    
    return PCA_values

In [55]:
PCA_session = pd.DataFrame()
for path in file_paths:
    int_data = analyze_PCA_from_med(path)
    PCA_session = pd.concat((PCA_session, int_data), axis = 0)


In [54]:
PCA_session

,lever_press,head_entries,lever_prob,head_prob,ave_lat_lever,ave_lat_head,response_bias,prob_diff,PCA_score,Date,Animal,Session
0,0,61,0.00,0.44,0.000000,2.385925,-1.000000,-0.44,-0.559531,180628,266,Pav7
0,7,2,0.16,0.08,1.752711,1.274524,0.555556,0.08,0.227791,180628,268,Pav7
0,0,12,0.00,0.28,0.000000,1.468501,-1.000000,-0.28,-0.475617,180628,269,Pav7
0,1,42,0.04,0.56,0.373410,1.800542,-0.953488,-0.52,-0.538734,180628,270,Pav7
0,0,11,0.00,0.20,0.000000,1.855440,-1.000000,-0.20,-0.461848,180628,271,Pav7
0,0,11,0.00,0.08,0.000000,1.835910,-1.000000,-0.08,-0.421197,180628,273,Pav7
0,5,13,0.08,0.24,1.727089,2.385199,-0.444444,-0.16,-0.223418,180628,274,Pav7
0,0,1,0.00,0.04,0.000000,0.823522,-1.000000,-0.04,-0.374117,180628,275,Pav7
0,2,21,0.08,0.52,1.278090,1.938294,-0.826087,-0.44,-0.444036,180628,277,Pav7
0,0,3,0.00,0.08,0.000000,2.642411,-1.000000,-0.08,-0.448080,180628,278,Pav7


In [ ]:
i = 0
for path in file_paths:
    print(i)
    data_frame = analyze_PCA_from_med(path)
    print(data_frame)
    i = i + 1

In [ ]:
array = list(map(make_array_from_path, file_paths))
array[0]